# Paragraph Vector (Doc2Vec)

_**Paragraph Vector**_ foi criado por Le e Mikolov (2014) e é uma estrutura não supervisionada que aprende representações vetoriais distribuídas para textos. Os textos podem ser de tamanho variável, variando de frases a documentos. No modelo, a representação vetorial é treinada para predizer palavras em um parágrafo. Mais precisamente, concatena-se o vetor do parágrafo com vários vetores de palavras presentes no parágrafo, com o objetivo de predizer a próxima palavra no contexto dado.

Tanto os vetores de palavras, quanto os de parágrafo são treinados pela descida de gradiente estocástica e pós-propagação (Rumelhart et al., 1986). Enquanto os vetores de parágrafo são únicos entre os parágrafos, os vetores de palavras são compartilhados (o vetor de uma palavra é o mesmo para todos os parágrafos que possuem aquela palavra). No momento da predição, os vetores de parágrafo são inferidos corrigindo os vetores de palavra e treinando o novo vetor de parágrafo até a convergência. Os autores propuseram dois algoritmos para a geração de vetores de parágrafo:

* **PV-DM** (_Distributed Memory Model of Paragraph Vectors_): Neste modelo, cada parágrafo é mapeado para um vetor exclusivo, representado por uma coluna em uma matriz $D$. Cada palavra também é mapeada para um vetor exclusivo, representado por uma coluna em uma matriz $W$. A concatenação ou média do vetor de parágrafo com os vetores de palavras são utilizados para prever a próxima palavra em um contexto. O vetor de parágrafo pode ser considerado uma pseudo-palavra e representa as informações que faltam no contexto atual, atuando como uma memória do tópico do parágrafo.

<img src="images/pv_dm.png" width="500">

* **PV-DBOW** (_Distributed Bag of Words version of Paragraph Vector_): Neste modelo, as palavras de contexto são ignoradas na entrada e previstas aleatoriamente na saída a partir do vetor do parágrafo.

<img src="images/pv_dbow.png" width="500">

Segundo Mikolov e Le (2014), cada vetor de parágrafo é uma combinação de dois vetores: um aprendido pelo PV-DM e outro aprendido pelo PVDBOW. O PV-DM sozinho geralmente funciona bem para a maioria das tarefas, mas sua combinação com o PV-DBOW é mais consistente em muitas tarefas e, portanto, altamente recomendado.

Neste notebook, treinaremos um modelo Doc2Vec em um córpus de sinopses de filmes.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucasosouza/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import gensim.models as g
import logging
from nltk.corpus import stopwords

In [3]:
#Save dataset/model
!mkdir data
!mkdir model
!mkdir "model/doc2vec"

mkdir: data: File exists
mkdir: model: File exists
mkdir: model/doc2vec: File exists


Aqui definimos as varíaveis que serão usadas como parâmetro no nosso treinamento.

In [4]:
#doc2vec parameters
vector_size = 300
window_size = 15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100 #100
dm = 1 #0 = dbow; 1 = dmpv
worker_count = 1 #number of parallel processes

O córpus que utilizaremos é um de sinopses de filmes com aproximadamente 10496 sinopses.

In [5]:
#input corpus
train_corpus = "data/sinopses.txt" #train_docs

#output model
saved_path = "model/doc2vec/model.bin"

Faremos a remoção de _stopwords_ para eliminar ruído dos dados.

In [6]:
#stopwords
pt_stop = stopwords.words('portuguese')
pt_stop.extend(['para','que'])
print(pt_stop)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos

Esse método faz a leitura do córpus enquanto remove as _stopwords_.

* **TaggedDocument**: Adiciona uma _label_ para cada documento (sinopse). Neste caso, estamos adicionando o contador como _label_. Normalmente se usa um id inteiro único.

In [7]:
def read_corpus(fname, tokens_only=False): 
    with open(fname) as f: #
        read_data = f.readlines()
        for i, line in enumerate(read_data):
            t_corpus = line.split(" | ")
            if(len(t_corpus)==2):
                #removing stopwords
                words = t_corpus[1].split()
                filtered_words = [word for word in words if word not in pt_stop]
                yield g.doc2vec.TaggedDocument(filtered_words, [i])

In [8]:
train_docs = list(read_corpus(train_corpus))

In [9]:
train_docs[:5]

[TaggedDocument(words=['As', 'coisas', 'mal', 'Inteligência', 'Britânica,', 'pois', 'Smersh', 'começara', 'sabotar', 'estabilidade', 'global:', 'nada', 'menos', 'onze', 'agentes', 'abatidos', 'e,', 'piorar', 'coisas,', 'maior', 'agente', 'secreto,', '007,', 'desfrutando', 'aposentadoria.', 'Sir', 'James', 'Bond,', 'primeiro', '007,', 'é', 'convencido', 'alguns', 'chefes', 'agências', 'espionagem', 'combater', 'inimigo', 'comum.', 'Essa', 'versão', '"Cassino', 'Royale"', 'é', 'versão', 'oficial', 'filmes', '007,', 'pois', 'rodado', 'outra', 'equipe,', 'estúdios,', 'padrões', 'contratos.', 'É', 'produção', 'anglo-americana', '1967,', 'gênero', 'comédia', 'espionagem.'], tags=[0]),
 TaggedDocument(words=['A', 'ação', 'é', 'eletrizante', 'ininterrupta,', 'pois', 'agente', '007', '(Sean', 'Connery)', 'vai', 'além', 'dever', 'ofício', 'profundezas', 'oceano', 'encontrar', 'perigoso', 'criminoso', 'ameaçando', 'milhões', 'pessoas', 'através', 'chantagem', 'destruir', 'mundo', 'meio', 'holocau

In [10]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Parâmetros

Na próxima célula de código, definimos os seguintes parâmetros:
* **size**: dimensionalidade dos vetores.

* **window**: é a quantidade de palavras anteriores e posteriores à palavra alvo.

* **min_count**: ignore as palavras com frequência total inferior a **min_count**.

* **sample**: limiar para configurar quais palavras de maior frequência são aleatoriamente reduzidas; O padrão é 1e-3, o intervalo útil é (0, 1e-5).

* **workers**: parâmetro que indica quantos cores da máquina serão utilizados para o treinamento.

* **hs**: se 1, softmax hierárquico será usado para o treinamento do modelo. Se definido como 0 (padrão), e existir amostragem negativa, esse recurso será utilizado.

* **dm**: define o algoritmo de treinamento. Por padrão, o DBOW é usado (dm = 0). O outro é o DMPV (dm = 1).

* **negative**: se > 0, será utilizada amostragem negativa. O valor indica quantas "palavras de ruído" devem ser consideradas (normalmente entre 5 a 20). Se **negative** configurado para 0, não é utilizada a amostragem negativa.

* **dbow_words**: se 1, skip-gram é usado para gerar os vetores de palavras simultaneamente com DBOW; O padrão é 0. Essa funcionalidade aumenta o conjunto de dados ao adicionar os vetores de palavras junto aos de documento. o treinamento ficará mais lento.

* **dm_concat**: se 1, usa a concatenação de vetores de contexto em vez da soma/média; O padrão é 0 (desativado).

* **iter**: número de iterações (épocas) sobre o córpus. O padrão é 5.





In [11]:
model = g.doc2vec.Doc2Vec(size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)

/Users/lucasosouza/anaconda/envs/udacity/lib/python3.5/site-packages/gensim/models/doc2vec.py:355: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Users/lucasosouza/anaconda/envs/udacity/lib/python3.5/site-packages/gensim/models/doc2vec.py:359: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-02-08 14:32:10,172 : INFO : using concatenative 9300-dimensional layer1


* **build_vocab**: Método que constrói um dicionário de palavras distintas presentes no córpus.

In [12]:
model.build_vocab(train_docs)

2018-02-08 14:32:10,423 : INFO : collecting all words and their counts
2018-02-08 14:32:10,424 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-02-08 14:32:10,529 : INFO : collected 62995 word types and 7771 unique tags from a corpus of 7529 examples and 296335 words
2018-02-08 14:32:10,529 : INFO : Loading a fresh vocabulary
2018-02-08 14:32:10,764 : INFO : min_count=1 retains 62995 unique words (100% of original 62995, drops 0)
2018-02-08 14:32:10,765 : INFO : min_count=1 leaves 296335 word corpus (100% of original 296335, drops 0)
2018-02-08 14:32:10,961 : INFO : deleting the raw counts dictionary of 62995 items
2018-02-08 14:32:10,963 : INFO : sample=1e-05 downsamples 5677 most-common words
2018-02-08 14:32:10,964 : INFO : downsampling leaves estimated 164586 word corpus (55.5% of prior 296335)
2018-02-08 14:32:11,162 : INFO : estimated required memory for 62995 words and 300 dimensions: 2459830700 bytes
2018-02-08 14:32:11,163 : INFO : resetting

Aqui ocorre o treinamento do nosso modelo.

In [13]:
#train doc2vec model
%time model.train(train_docs, total_examples=model.corpus_count, epochs=model.iter)

/Users/lucasosouza/anaconda/envs/udacity/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if __name__ == '__main__':
2018-02-08 14:32:12,317 : INFO : training model with 1 workers on 62996 vocabulary and 9300 features, using sg=0 hs=0 sample=1e-05 negative=5 window=15
2018-02-08 14:32:13,407 : INFO : EPOCH 1 - PROGRESS: at 6.64% examples, 10540 words/s, in_qsize 1, out_qsize 0
2018-02-08 14:32:14,656 : INFO : EPOCH 1 - PROGRESS: at 16.43% examples, 12288 words/s, in_qsize 1, out_qsize 0
2018-02-08 14:32:15,697 : INFO : EPOCH 1 - PROGRESS: at 26.98% examples, 13628 words/s, in_qsize 1, out_qsize 0
2018-02-08 14:32:16,958 : INFO : EPOCH 1 - PROGRESS: at 40.63% examples, 14865 words/s, in_qsize 1, out_qsize 0
2018-02-08 14:32:18,211 : INFO : EPOCH 1 - PROGRESS: at 55.17% examples, 15668 words/s, in_qsize 1, out_qsize 0
2018-02-08 14:32:19,458 : INFO : EPOCH 1 - PROGRESS: at 6

CPU times: user 13min 4s, sys: 2.97 s, total: 13min 7s
Wall time: 13min 7s


Iremos salvá-lo em disco para uso no próximo notebook.

In [14]:
#save model
model.save(saved_path)

2018-02-08 14:45:19,657 : INFO : saving Doc2Vec object under model/doc2vec/model.bin, separately None
2018-02-08 14:45:19,658 : INFO : storing np array 'vectors' to model/doc2vec/model.bin.wv.vectors.npy
2018-02-08 14:45:19,772 : INFO : storing np array 'syn1neg' to model/doc2vec/model.bin.trainables.syn1neg.npy
2018-02-08 14:45:24,709 : INFO : saved model/doc2vec/model.bin
